# Übungsblatt 8 - Programmieraufgaben (15 Punkte)
Einführung in Deep Learning für Visual Computing

**Deadline : 18.06.2025 - 14:00 via VIPS**

## Import

In [ ]:
from typing import Type, Tuple, List

import os
import time
import torch
import torch.nn as nn
import torchvision
import numpy as np


from visualize import (
    show_image_grid,
    show_loss_curve,
    show_distribution_samples
)

from training import (
    train_multiclass,
    evaluate
)

# Transfer Learning (8 Punkte)

Das Packet `Torchvision` stellt nicht nur Datensätze und Werkzeuge für das Training von Computer Vision bereit sondern auch Implementierung populärer Neuronale Netzwerke für Computer Vision. In dieser Aufgabe verwenden wir die Torchvision Implementierung von ResNet18, der kleinsten ResNet-Konfiguration, die von den Ursprünglichen Autoren untersucht wurde. Wir erhalten eine Instanz des Modells mit der Funktion

`python
resnet18 = torchvision.models.resnet18().
`

Die Funktion gibt die Instanz eines Netzwerks zurück, die aus den uns geläufigen Klassen und Funktionen in `torch.nn` aufgebaut ist. Wir können das Netzwerk verwenden wie wir es gewohnt sind. Torchvision stellt aber nicht nur die Implementierung des Netzwerks bereit, sondern auch die Netzwerkgewichte des auf ImageNet trainierten Netzwerks. Um die Gewichte zu laden, wird der Funktion ``resnet18` die gewünschten Gewichte übergeben mit

`python
resnet18 = torchvision.models.resnet18(torchvision.models.ResNet18_Weights.IMAGENET1K_V1).
`

Nach dem Erzeugen des Modells können wir auf die einzelnen Schichten des ResNet18 Modells wie folgt zugreifen:

`python
resnet18.<layer_name>
`

Die Layer des Netzwerks sind - in Reihenfolge in der Sie in der forward Methode aufgerufen werden:
* `resnet18.conv1`
* `resnet18.bn1`
* `resnet18.relu`
* `resnet18.maxpool`
* `resnet18.layer1` (Vereint mehrere ResNet Blöcke)
* `resnet18.layer2` (Vereint mehrere ResNet Blöcke)
* `resnet18.layer3` (Vereint mehrere ResNet Blöcke)
* `resnet18.layer4` (Vereint mehrere ResNet Blöcke)
* `resnet18.avgpool`

Mit Hilfe von `nn.Sequential(resnet18.conv1,resnet18.bn1,...)` können wir eine beliebige Anzahl dieser vortrainierten Layers wieder zusammenfassen und als Basis für ein neues Netzwerk verwenden. Diese vortrainierten Layer können z.B. in Kombination mit anschließenden (noch untrainierten) Linear Layers dann verwendet werden um auf neue Probleme trainiert und angewandt zu werden. Mit Hilfe von `self.layerX.requires_grad_(train_features)` kann angegeben werden, ob ein layer beim Training mitberücksichtigt oder als statisch betrachtet werden soll. In der folgenden Aufgabe werden wir verschiedene Konfiguationsmöglichkeiten für solche Transfer Learning Verfahren betrachten.

**Aufgabe :**
1. (1 Bonus-Punkt) Vervollständigen Sie die Klasse `ResNetBlock` sodass diese einen Block aus zwei Conv.-Layers mit einer Skip-Connections implementiert, ähnlich zu Folie 25ff in Vorlesung 7. Verwenden Sie die ReLU-Funktion als Aktivierungsfunktion.

2. (3 Punkte) Vervollständigen Sie die Klasse `PretrainedClassifier`, sodass diese das auf ImageNet vortrainierte ResNet18 verwendet um Feature aus den Eingabebildern zu extrahieren. Die Features sollen anschließend durch das Modul `classifier` auf 10 Klassen abgebildet werden. Wählen Sie die Schichten des Resnet aufbauend auf dem Wissen über hierarchischen CNN-Architekturen aus der Vorlesung. Geben Sie eine kurze Begründung Ihrer Auswahl.

2. (4 Punkte) Trainieren Sie drei unterschiedliche Konfigurationen des Netzwerks `PretrainedClassifier`, wobei jede Konfiguration jeweils nur eine Epoche auf dem CIFAR10-Datensatz durchläuft. Die Konfigurationen sind gegeben als:
    * In der ersten Konfiguration sollen die Gewichte des Modells zufällig initialisiert werden, und es sollen alle Parameter des Netzwerks trainiert werden.
    * In der zweiten Konfiguration sollen die Gewichte des ResNet18 auf ImageNet vortrainiert sein, und es sollen nur die Parameter der neu hinzugefügten Layer trainiert werden.
    * In der dritten Konfiguration sollen die Gewichte des ResNet18 auf ImageNet vortrainiert sein, und es sollen alle Parameter des Netzwerks trainiert werden.

Bestimmen Sie anschließend die Genauigkeit, die auf den Testdaten des Datensatzes erreicht wird. Vergleichen Sie die Laufzeit und die Genauigkeit der drei Konfigurationen untereinander.

**Hinweis :** Sie die Ihnen bereits bekannten Funktionen `train_multiclass` und `evaluate` verwenden.

In [ ]:
class PretrainedClassifier(nn.Module):
    def __init__(self, pretrained : bool, train_features : bool) -> None:
        super().__init__()
        self.train_features = train_features
        resnet18 = torchvision.models.resnet18(torchvision.models.ResNet18_Weights.IMAGENET1K_V1 if pretrained else None)
        # Beginn Iher Lösung
        # Ende Ihrer Lösung

    def forward(self, x : torch.Tensor) -> torch.Tensor:
        self.features.train(self.train_features)
        f = self.features(x)
        #print(f.shape)
        return self.classifer(f)

class ResNetBlock(nn.Module):
    def __init__(self, num_channels : int, kernel_size : int):
        super().__init__()
        # Beginn Ihrer Lösung
        # Ende Ihrer Lösung

    def forward(self, x):
        # Beginn Ihrer Lösung
        pass # Ersetzen Sie `pass` durch Ihre Lösung
        # Ende Ihrer Lösung

In [ ]:
transforms = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
])
cifar10_train = torchvision.datasets.CIFAR10("./", True, transform=transforms, download=True)
dataloader = torch.utils.data.DataLoader(cifar10_train, 128, True)
cifar10_test = torchvision.datasets.CIFAR10("./", False, transform=transforms, download=True)
dataloader_test = torch.utils.data.DataLoader(cifar10_test, 128, True)

# Training und Evaluation der Modelle
# Beginn Ihrer Lösung
# Ende Ihrer Lösung

# Generative Adversarial Networks (7 Punkte)

**Aufgabe :**

1. (5 Punkte) Implementieren Sie den Algorithmus zum Trainieren eines GANs, wie er in Vorlesung 9, Folie 19 beschrieben ist, indem Sie die Funktion `train_gan` vervollständigen. Setzen Sie bei der Implementierung des Algorithmus $k=1$. Das Training soll für `num_epochs` durchgeführt werden. Am Ende jeder Epoche sollen 25 Stichproben (Bilder) mit dem Generator erzeugt und der Liste `img_list` hinzugefügt werden. Darüber hinaus soll die Funktion eine Liste der Zielfunktionswerte des Generators und des Discriminators, die während des Trainings auftreten, zurückgeben.

2. (2 Punkte) Trainieren Sie ein GAN auf dem FashionMNIST-Datensatz für 10 Epochen und erstellen Sie einen Plot der Zielfunktion des Generators und des Discriminators. Zeigen Sie 25 Bilder des Trainingsdatensatzes sowie den Verlauf der 25 Stichproben aus der gelernten Verteilung, die am Ende jeder Epoche erzeugt wurden, mit Hilfe der Funktion `show_image_grid` an.

Hinweis: Da das Training eines GANs instabil sein kann, müssen Sie möglicherweise das Training mehrfach wiederholen oder die Hyperparameter anpassen, bis das Netzwerk konvergiert.

In [ ]:
# Bildet einen Tensor [50, 1, 1] auf ein Bild [1,28,28] ab
class Generator(nn.Module):

    def __init__(self):
        super().__init__()

        self.model = nn.Sequential(
            # Block 1
            nn.ConvTranspose2d(50, 64, 4, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            # Block 2
            nn.ConvTranspose2d(64, 64, 4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            # Block 3
            nn.ConvTranspose2d(64, 32, 4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            # Output
            nn.ConvTranspose2d(32, 1, 4, stride=2, padding=3, bias=False),
            nn.Tanh()
        )

    def forward(self, z):
        return self.model(z)

# Ein Bild auf ein Bild [1,28,28] auf einen Tensor [1,1,1] ab
class Discriminator(nn.Module):

    def __init__(self):
        super().__init__()

        self.model = nn.Sequential(
            # Block 1
            nn.Conv2d(1, 32, 4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(32),
            nn.LeakyReLU(negative_slope=0.2),
            # Block 2
            nn.Conv2d(32, 64, 4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(negative_slope=0.2),
            # Block 3
            nn.Conv2d(64, 64, 4, bias=False),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(negative_slope=0.2),
            # Output
            nn.Conv2d(64, 1, 4, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

def train_gan(num_epochs, dataloader, netD, optimizerD, netG, optimizerG):
    G_losses, D_losses, img_list= [], [], []

    # Beginn Ihrer Lösung
    # Ende Ihrer Lösung
    return G_losses, D_losses, img_list

netD = Discriminator()
netG = Generator()

# Setup Adam optimizers for both G and D
optimizerD = torch.optim.Adam(netD.parameters(), lr=0.0005, betas=(0.5, 0.999))
optimizerG = torch.optim.Adam(netG.parameters(), lr=0.0003, betas=(0.5, 0.999))

traindata = torchvision.datasets.FashionMNIST("./data", train=True, transform=torchvision.transforms.ToTensor(), download=True)
trainloader = torch.utils.data.DataLoader(traindata, batch_size=128, shuffle=True)
G_losses, D_losses, img_list = train_gan(10, trainloader, netD, optimizerD, netG, optimizerG)

# Visualisierung der Zielfunktionskurve und Bild
# Begin Ihrer Lösung
# Ende Ihrer Lösung
